### GridBatch and JaggedTensor
There are two fundamental classes in fVDB you will encounter frequently: `GridBatch` and `JaggedTensor`.

```python
fvdb.GridBatch
fvdb.JaggedTensor
```

#### GridBatch

A `GridBatch` is an indexing structure which maps the 3D $ijk$ coordinates of a set of **sparse** grids to integer offsets which can be used to look up attributes in a tensor that correspond with each voxel.

This mapping only exists for $ijk$ coordinates which are **active** in the space of a **sparse**, 3D grid.

&#x1F4A1; We call these 3D grids **sparse** because of this arbitrary nature to the topology of **active** voxels in the grid compared to **dense** grids where all voxels exist inside some chosen extents in each dimension.

The figure below illustrates this $ijk$ mapping process for a `GridBatch` containing only a single grid.


<center>

<img src="img/gridbatch.png"  alt="Image Index Grid" width="800"/>

</center>

In practice, `GridBatch` is an ordered collection of 1 or more of these 3D grids.  

&#x1F529;	At the level of technical implementation, these 3D grids are **NanoVDB** grids of the special `IndexGrid` type which only stores a unique **index** integer value at each active voxel location.  This **index** is an offset into some external data array, a tensor, (one that is not contained in the `IndexGrid`/`GridBatch` classes) where contiguous array members correspond to spatially nearby voxels.  `IndexGrid` will allow us to reference into this 'sidecar' tensor of attribute data given a spatial $ijk$ set of coordinates.

Each grid member in a `GridBatch` can have different topologies, different numbers of active voxels and different voxel dimensions and origins per-grid.

<center>

<img src="img/gridbatch_concept.svg"  alt="Image Index Grid" width="800"/>

</center>

##### Images as 2D GridBatch


To help explain these concepts, let's consider how we might treat image data with this framework (an image can be thought of as a dense 2D grid after all).  

If an image is a 2D grid of pixels, we could imagine the position of each pixel could be expressed as $i,j$ coordinates.  For an RGB image, each pixel would contain 3 associated values $(R,G,B)$.

In this way, we could decompose an image into:

1.  an `IndexGrid` of $i,j$ coordinates
2.  a tensor of size $[NumberPixels, 3]$ (the flattened RGB features for our attribute tensor)

The $i,j$ coordinates can be used with the `IndexGrid` to index into the list of RGB values to retrieve the RGB value for each pixel.  

<center>

<img src="img/image_index_grid_diagram.svg"  alt="Image Index Grid" width="1000"/>

</center>

If we had many images, each of different sizes, we can imagine constructing a `GridBatch` as an ordered set of their `IndexGrid`s.

All the RGB values would go into a sidecar attribute tensor where each feature array would have to be of a different length corresponding to the image size.  We call this attribute list of different-length features a **jagged tensor**.

<center>

<img src="img/grid_batch_diagram.svg"  alt="Grid Batch" width="600"/>

</center>

This is the essence of the relationship between `GridBatch` and `JaggedTensor` in fVDB, but more on `JaggedTensor` later…

Lastly, it is important to know that each grid in the `GridBatch` will be on the same device and processed together by operators in mini-batch-like fashion.  

Let's put together our first `GridBatch` to see how it works.

In [1]:
# import the usual suspects and fvdb
import numpy as np
import torch
import fvdb

We will make a `GridBatch` of 8 grids, each with a different number of active voxels and all of those voxels' positions being chosen randomly.  Further, each grid will have a randomly chosen origin in the 3D world space and a randomly chosen voxel size.

In [2]:
batch_size = 8
# Randomly generate different numbers of voxels we desire in each grid in our batch
num_voxels_per_grid = [np.random.randint(100, 1_000) for _ in range(batch_size)]

# A list of randomly generated 3D indices for each grid in our batch in the range [-512, 512]
ijks = [torch.randint(-512, 512, (num_voxels_per_grid[i], 3), device='cuda') for i in range(batch_size)]

# Create an fvdb.GridBatch from the list of indices!!
grid_batch = fvdb.sparse_grid_from_ijk(fvdb.JaggedTensor(ijks), # We'll explain JaggedTensor in a moment…
                                        voxel_sizes = [np.random.rand(3).tolist() for _ in range(batch_size)], # Random, different voxel sizes for each grid in our batch
                                        origins     = [np.random.rand(3).tolist() for _ in range(batch_size)], # Random, different grid origins for each grid in our batch
                                      )

There are many more convenient ways that fVDB provides to create a `GridBatch` besides building from lists of coordinate indexes such as building a `GridBatch` from **worldspace pointclouds, meshes or dense tensors**.

&#x1F4A1; The fVDB documentation has more useful examples for these cases using functions like `sparse_grid_from_points`, `sparse_grid_from_dense` and `sparse_grid_from_mesh`.

In [3]:
# This grid will be on the GPU because the `ijks` were on that device
assert(grid_batch.device == ijks[0].device == torch.device('cuda:0'))

Each member of the batch has different voxel size dimensions, a different origin in space and different number of voxels

In [4]:
for i in range(grid_batch.grid_count):
    print(f"""Grid {i} has {grid_batch.num_voxels_at(i)} voxels,
        voxel size of {grid_batch.voxel_size_at(i).tolist()}
        and an origin of {grid_batch.origin_at(i).tolist()}""")

Grid 0 has 116 voxels,
        voxel size of [0.9519320130348206, 0.9454407095909119, 0.36045560240745544]
        and an origin of [0.49334126710891724, 0.26929008960723877, 0.4102530777454376]
Grid 1 has 768 voxels,
        voxel size of [0.19912821054458618, 0.008141621947288513, 0.9990909695625305]
        and an origin of [0.6951860189437866, 0.5513076782226562, 0.8320647478103638]
Grid 2 has 920 voxels,
        voxel size of [0.6421335935592651, 0.47253838181495667, 0.9842809438705444]
        and an origin of [0.43986809253692627, 0.8472382426261902, 0.4374838173389435]
Grid 3 has 930 voxels,
        voxel size of [0.2742624282836914, 0.10442875325679779, 0.7849230766296387]
        and an origin of [0.2953282296657562, 0.6793888807296753, 0.5144127011299133]
Grid 4 has 981 voxels,
        voxel size of [0.9298512935638428, 0.5107961893081665, 0.08465440571308136]
        and an origin of [0.6837033629417419, 0.9869440793991089, 0.8731261491775513]
Grid 5 has 627 voxels,
       

Let's examine some of the ways we can retrieve indices from a `GridBatch` based on $ijk$ coordinates.

In [5]:
# Let's retrieve a random ijk coordinate from each of the lists we used to make the grids
ijk_queries = fvdb.JaggedTensor([ijks[n][np.random.randint(len(ijks[n]))][None,:] for n  in range(grid_batch.grid_count)])

# Use the GridBatch to get indices into the sidecar feature array from the `ijk` coordinate in each grid
feature_indices = grid_batch.ijk_to_index(ijk_queries)
world_positions = grid_batch.grid_to_world(ijk_queries.float())
for i, (ijk, world_p, i_f) in enumerate(zip(ijk_queries, world_positions, feature_indices)):
    print(f"Grid {i}, feature Index at ijk {ijk.jdata.tolist()}, world-space {world_p.jdata.tolist()} : {i_f.jdata.item()}")

# NOTE: This GridBatch (Batch of IndexGrids) just expresses the topology of the grids and can be used to reference a sidecar flat array of features but we won't create this sidecar in this example…
# We can get the index into this hypothetical sidecar feature array with any `ijk` coordinate (if we ask for an `ijk` not in the grid, -1 is given as the index)

Grid 0, feature Index at ijk [[-283, -242, -57]], world-space [[-268.9034118652344, -228.52735900878906, -20.13571548461914]] : 4
Grid 1, feature Index at ijk [[466, 245, 313]], world-space [[93.48892974853516, 2.5460050106048584, 313.5475158691406]] : 756
Grid 2, feature Index at ijk [[-151, 193, -299]], world-space [[-96.52230072021484, 92.0471420288086, -293.8625183105469]] : 286
Grid 3, feature Index at ijk [[302, -130, 70]], world-space [[83.12258911132812, -12.89634895324707, 55.45902633666992]] : 681
Grid 4, feature Index at ijk [[423, -340, -478]], world-space [[394.01080322265625, -172.6837615966797, -39.591678619384766]] : 590
Grid 5, feature Index at ijk [[-199, -45, -259]], world-space [[-88.95337677001953, -28.754833221435547, -203.76846313476562]] : 58
Grid 6, feature Index at ijk [[-163, 121, 189]], world-space [[-158.69960021972656, 53.49298858642578, 136.29576110839844]] : 403
Grid 7, feature Index at ijk [[493, -252, 261]], world-space [[489.8296813964844, -81.9899978

#### JaggedTensor

`JaggedTensor` is the supporting feature data (i.e. the 'sidecar' of attributes) that is paired with a `GridBatch`.  

You can think of `JaggedTensor` as an ordered list of PyTorch Tensors, one for each grid in the `GridBatch`.  Same as `GridBatch`, the Tensors are all on the same device and processed together in a mini-batch. 

In [6]:
# Here we create a list of Tensors, one for each grid in the batch with its number of active voxels and 8 random features per-voxel
list_of_features = [torch.randn(int(grid_batch.num_voxels[i]), 5, device=grid_batch.device) for i in range(grid_batch.grid_count)]
# Now we make a JaggedTensor out of this list of heterogeneous Tensors
features  = fvdb.JaggedTensor(list_of_features)

In the `JaggedTensor` above, you can see how we constructed it with a list of heterogeneously sized Tensors whose shapes were of the form:

$$[ [B1, C], [B2, C], [B3, C], …] ]$$

where the value of $Bn$ would be the number of active voxels in each grid of the `GridBatch` and $C$ is the number of feature channels (5 was chosen in this case).

&#x1F4A1; Note how each Tensor element in our `JaggedTensor` can have different numbers of active voxels (similar to `GridBatch`) but the same number of per-voxel feature channels.  This is distinctly different from the classic representation of 3D data in a PyTorch `Tensor` which is usually a homogeneously shaped Tensor of shape $[N, C, H, W, D]$ where $N$ would be the number of "grids" in our batch, $C$ is the number of feature channels, and $H, W, D$ are the 3 index dimensions of a **dense** grid.

We can also directly use a `GridBatch` to derive a `JaggedTensor` to match the `GridBatch`'s specific sizing.

This more concise line of code has the same effect as the previous example:

In [7]:
# From a GridBatch, a matching JaggedTensor can be created from a Tensor of shape `(total_voxels, feature_dim)`
features = grid_batch.jagged_like(torch.randn(grid_batch.total_voxels, 5, device=grid_batch.device))

Now that we have a `GridBatch` and a `JaggedTensor` of attributes that correspond to the batch of grids, we could use the $ijk$ offsets we can obtain from the `GridBatch` to index into the `JaggedTensor` to retrieve the feature data for the voxels at those $ijk$'s.

In [8]:
features[feature_indices].jdata

tensor([[-1.4529,  1.5827, -0.5729, -0.5724, -0.0612],
        [-1.4506, -0.9575,  0.0528, -1.1230,  2.0563],
        [-2.3013,  1.2390,  0.8193,  0.1099,  1.1040],
        [-0.9724, -0.9775, -0.3134,  0.7936, -1.3263],
        [ 2.7002,  0.3406, -0.3990,  2.0181, -0.1572],
        [-1.1491,  0.8803,  0.1625,  1.7068,  0.4811],
        [ 1.8915,  0.3565, -0.3147,  1.0703, -0.1139],
        [-1.1003, -1.2272,  0.8648, -0.4511, -0.3558]], device='cuda:0')

But beyond just looking up the features directly, we can use higher-level ƒVDB functionality like sampling what the feature values are at a given worldspace position.

Let's get the feature values at the worldspace positions from earlier in the lesson:

In [9]:
sampled_features = grid_batch.sample_trilinear(world_positions, features)


for xyz, value in zip(world_positions.jdata, sampled_features.jdata):
    print(f"World position {[round(num, 1) for num in xyz.tolist()]} \n\thas the feature values: {[round(num, 3) for num in value.tolist()]}")

World position [-268.9, -228.5, -20.1] 
	has the feature values: [-1.453, 1.583, -0.573, -0.572, -0.061]
World position [93.5, 2.5, 313.5] 
	has the feature values: [-1.451, -0.957, 0.053, -1.123, 2.056]
World position [-96.5, 92.0, -293.9] 
	has the feature values: [-2.301, 1.239, 0.819, 0.11, 1.104]
World position [83.1, -12.9, 55.5] 
	has the feature values: [-0.972, -0.977, -0.313, 0.794, -1.326]
World position [394.0, -172.7, -39.6] 
	has the feature values: [2.7, 0.341, -0.399, 2.018, -0.157]
World position [-89.0, -28.8, -203.8] 
	has the feature values: [-1.149, 0.88, 0.162, 1.707, 0.481]
World position [-158.7, 53.5, 136.3] 
	has the feature values: [1.891, 0.356, -0.315, 1.07, -0.114]
World position [489.8, -82.0, 108.2] 
	has the feature values: [-1.1, -1.227, 0.865, -0.451, -0.356]


### JaggedTensor Implementation Details


Internally, `JaggedTensor` does not represent the list of features as a list of differently sized PyTorch tensors, but instead stores a single tensor of the features accompanied by a special index structure that allows us to access the feature data for each grid in the `GridBatch`.  

The `jdata` attribute of a `JaggedTensor` contains all the feature data values in this single list.  `jdata` is a `Tensor` of shape $[N, C]$ where $N$ is the total number of active voxels in the batch and $C$ is the number of feature channels.  

`jdata`'s shape would be equivalent to the result of concatenating the list of heterogeneously sized Tensors, mentioned above, along their first axis into a single Tensor whose shape would be $[B1+B2+B3…+Bn, C]$.

<center>

<img src = "img/jdata.jpg" width=1200 alt="jdata">

</center>

In [10]:
print(f"size of features data for the entire GridBatch: {features.jdata.shape}")

size of features data for the entire GridBatch: torch.Size([6105, 5])


To determine which grid each feature belongs to, `JaggedTensor` contains indexing information in its `jidx` attribute.

`jidx` is a `Tensor` of shape $[N]$ where $N$ is again the total number of active voxels across the batch.  Each element of `jidx` is an integer that tells us which grid in the `GridBatch` the corresponding feature in `jdata` belongs to.  The grid membership in `jidx` is ordered starting from 0 and members of the same batch are contiguous.

<center>

<img src = "img/jidx.jpg" width=1200 alt="jdata">

</center>

In [11]:
print(f"per-feature membership for each voxel in the GridBatch: {features.jidx}")
print(f"\nthe size of the features of the 4th grid in this batch: {features.jdata[features.jidx==3].shape}")

per-feature membership for each voxel in the GridBatch: tensor([0, 0, 0,  ..., 7, 7, 7], device='cuda:0', dtype=torch.int32)

the size of the features of the 4th grid in this batch: torch.Size([930, 5])


Additionally, `JaggedTensor` has a `joffsets` attribute that can also be used to index into `jdata` to get the feature data for each grid in the batch.

The `joffsets` attribute is a `Tensor` of shape $[B]$, where $B$ is the number of grids in the batch.  `joffset`'s values are the start offsets into `jdata` that corresponds to each grid in the batch.  This is essentially the same information that can be found in `jidx` but expressed in a different form.

<center>

<img src = "img/joffsets.jpg" width=1200 alt="jdata">

</center>

In [12]:
print("per-grid offsets into the feature data:")
print(features.joffsets)
print(f"\nthe size of the features of the 4th grid in this batch: {features.jdata[features.joffsets[3]:features.joffsets[3+1]].shape}")

per-grid offsets into the feature data:
tensor([   0,  116,  884, 1804, 2734, 3715, 4342, 5265, 6105], device='cuda:0')

the size of the features of the 4th grid in this batch: torch.Size([930, 5])


The features stored in a `JaggedTensor` can be of any type that PyTorch supports, including float, float64, float16, bfloat16, int, etc., and we can have an arbitrary number of feature channels per voxel.  

For instance, there could be a `JaggedTensor` with 1 float feature that represents a signed distance field in each grid, or 3 float features that represent an RGB color in each voxel of the grids, or a 192 float feature that represents a learned feature vector of each voxel in each grids.

In [13]:
# A single scalar feature
features = grid_batch.jagged_like(torch.randn(grid_batch.total_voxels, 1, dtype=torch.float, device=grid_batch.device))

# Cast to a double
features = features.double()

# A JaggedTensor of 192 float features
features = grid_batch.jagged_like(torch.randn(grid_batch.total_voxels, 192, dtype=torch.float, device=grid_batch.device))